In [50]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
import numpy as np
import os
from sklearn.model_selection import train_test_split
import cv2
from sklearn.metrics import accuracy_score, f1_score

In [38]:
# Autoencoder 모델 정의
def create_autoencoder():
    input_img = layers.Input(shape=(360, 240, 1))
    
    # Encoder
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2), padding='same')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    autoencoder = models.Model(input_img, decoded)
    return autoencoder

# 데이터 로드 및 전처리
def load_data(directory):
    images = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            img = cv2.imread(img_path, 0)  # 그레이스케일로 변환
            # img = cv2.resize(img, (256, 256))  # 크기 조정
            img_array = np.array(img) / 255.0  # 정규화
            images.append(img_array)
    return np.array(images)

# 모델 학습
def train_model(model, train_data, val_data, epochs=25):
    model.compile(optimizer='adam', loss='mean_squared_error')
    
    history = model.fit(
        train_data, train_data,
        epochs=epochs,
        batch_size=32,
        shuffle=True,
        validation_data=(val_data, val_data)
    )
    
    return model, history

# 크랙 탐지
def detect_cracks(model, data, threshold):
    reconstructions = model(data)
    reconstructions = np.squeeze(reconstructions, axis=-1)
    mse = np.mean(np.square(data - reconstructions), axis=(1,2))
    return mse > threshold

# 학습
- 학습은 메모리 문제로 인하여 Colab에서 진행하였습니다.

In [2]:
crack_data = load_data("/Users/raphaelseo/Documents/projects/SPH/og_data/crack")

In [46]:
loaded_model = keras.models.load_model('autoencoder.h5')

In [4]:
normal_data = load_data("/Users/raphaelseo/Documents/projects/SPH/og_data/normal")

# 정상 이미지를 학습 및 검증 세트로 분할
train_data, val_data = train_test_split(normal_data, test_size=0.2, random_state=42)
val_reconstructions = loaded_model.predict(val_data)
val_reconstructions = np.squeeze(val_reconstructions, axis=-1)
val_mse = np.mean(np.square(val_data - val_reconstructions), axis=(1,2))
threshold = np.percentile(val_mse, 95)  # 예: 95 퍼센타일을 임계값으로 사용

2024-07-17 20:39:53.393031: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 735ms/step


# 이상치 검출 기준
- validation 데이터를 활용하여 임계값을 설정하였습니다.
- 임계값은 95퍼센타일로 설정하였습니다.
- 이는 정상 이미지에서 생성된 이미지들 중 오차범위가 상위 5%이상인 이미지를 이상치로 간주하겠다는 의미입니다.

In [47]:
normal_data.shape

(93, 360, 240)

In [6]:
# Combine normal_data and crack_data
combined_data = np.vstack((val_data, crack_data))

# Print the shape of the combined data to verify
print("Shape of combined data:", combined_data.shape)

# If you need to keep track of which images are normal and which are cracks,
# you can create a corresponding label array
labels = np.concatenate([np.full(len(val_data), False), np.full(len(crack_data), True)])

print("Total number of images:", len(combined_data))
print("Number of normal images:", len(val_data))
print("Number of crack images:", len(crack_data))


Shape of combined data: (126, 360, 240)
Total number of images: 126
Number of normal images: 19
Number of crack images: 107


In [53]:
crack_results = detect_cracks(loaded_model, combined_data, threshold)
print(f"Accuracy: {accuracy_score(labels, crack_results)}")
print(f"F1 Score: {f1_score(labels, crack_results)}")

Accuracy: 0.8571428571428571
F1 Score: 0.9090909090909091


## 결과
- 약 0.91의 F1 score를 기록하였습니다.